In [1]:
import pandas as pd
import time,csv,sys,pytz,os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select,WebDriverWait
import numpy as np
from datetime import date,datetime,timedelta

In [2]:
%load_ext dotenv
%dotenv

In [3]:
def logTime(startTime, currentTime):
    endTime = time.time()
    currentTime = currentTime or endTime
    hours, rem = divmod(endTime-currentTime, 3600)
    minutes, seconds = divmod(rem, 60)
    print("[Duration:{:0>2}:{:0>2}:{:05.2f}, ".format(int(hours),int(minutes),int(seconds)), end='')
    hours, rem = divmod(endTime-startTime, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Total:{:0>2}:{:0>2}:{:05.2f}] ".format(int(hours),int(minutes),seconds), end='')
    return endTime

In [4]:
def export(df, name, path = ''):
    now = lambda: datetime.now().astimezone().isoformat(timespec='seconds').replace('-','').replace(':','')
    fullPath = 'Scraped-Data/' + path
    Path(fullPath).mkdir(parents=True, exist_ok=True)
    df.to_csv(fullPath + '/' + name + '-' + now() + '.csv')

In [5]:
def login():
    url = 'https://leetcode.com/accounts/login'
    driver.get(url)
    WebDriverWait(driver, timeout=15).until_not(lambda x: x.find_element_by_class_name("spinner").is_displayed())
    try:
        emailInput = driver.find_element_by_id('id_login')
        passwordInput = driver.find_element_by_id('id_password')
        loginButton = driver.find_element_by_id('signin_btn')
        emailInput.send_keys(os.getenv('LEETCODE_USERNAME'))
        passwordInput.send_keys(os.getenv('LEETCODE_PASSWORD'))
        loginButton.click()
    except:
        pass
    finally:
        WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_class_name("trending-topic-container").is_displayed())

In [6]:
def openAndExportAllProblems():
    url = 'https://leetcode.com/problemset/all/'
    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_tag_name("select").is_displayed())
    select = Select(driver.find_element_by_tag_name("select"))
    selectLen = len(select.options)
    select.select_by_index(selectLen-1)
    script = '''
const rows = document.getElementsByClassName('reactable-data')[0].children;
const problems = [];
for (const row of rows) {
  const problem = [], cols = row.children;
  const frequency = parseFloat(row.getElementsByClassName('progress-bar')[0].style.width);
  const link = cols[2].querySelector('a').href;
  problem.push(cols[1].innerText.trim());
  problem.push(cols[2].innerText.trim());
  problem.push(cols[4].innerText.trim());
  problem.push(cols[5].innerText.trim());
  problem.push(frequency);
  problem.push(link);
  problems.push(problem);
}
return problems
    '''
    problemsData = driver.execute_script(script)
    problems = pd.DataFrame(problemsData, columns = ['Id','Title','Acceptance','Difficulty','Frequency','Link'])
    problems = problems.set_index('Id')
    export(problems, 'Problems')
    print("Exported %d Problems (100%%)" % len(problems))
    return problems

In [7]:
def getProblemsByTitle():
    return allProblems.reset_index().set_index('Title')

In [8]:
def openAndExportAllCompanies():
    url = 'https://leetcode.com/problemset/all/'
    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_tag_name("select").is_displayed())
    driver.find_element_by_css_selector('#expand-company .btn').click()
    script = '''
const buttons = document.getElementById('current-company-tags').children;
const companies = [];
for (const button of buttons) {
  const company = [];
  company.push(button.firstElementChild.innerText.trim());
  company.push(button.firstElementChild.nextElementSibling.innerText.trim());
  company.push(button.href);
  companies.push(company);
}
return companies
    '''
    companiesData = driver.execute_script(script)
    companies = pd.DataFrame(companiesData, columns = ['Name','Count','Link'])
    companies.index = companies.index + 1
    companies.index.name = 'Serial'
    export(companies, 'Companies')
    print("Exported %d Companies (100%%)" % len(companies))
    return companies

In [9]:
def getFirstOption():
    driver.find_element_by_id('react-select-2--value-item').click()
    firstOption = driver.execute_script("return (4-document.querySelectorAll('#react-select-2--list div[role=option]').length);")
    driver.find_element_by_id('react-select-2--value-item').click()
    return firstOption

In [10]:
def selectOption(n):
    driver.find_element_by_id('react-select-2--value-item').click()
    driver.find_element_by_id('react-select-2--option-' + str(n)).click()

In [11]:
def getCompanyProblems():
    script = '''
const rows = document.getElementsByClassName('reactable-data')[0].children;
const problems = [];
for (const row of rows) {
    const problem = [], cols = row.children;
    const tags = Array.from(cols[3].querySelectorAll('a')).map(t => t.innerText.trim()).join(', ');
    const frequency = parseFloat(row.getElementsByClassName('progress-bar')[0].style.width);
    const link = cols[2].querySelector('a').href;
    problem.push(cols[1].innerText.trim());
    problem.push(cols[2].innerText.trim());
    problem.push(tags);
    problem.push(cols[4].innerText.trim());
    problem.push(cols[5].innerText.trim());
    problem.push(frequency);
    problem.push(link);
    problems.push(problem);
}
return problems
    '''
    companyProblemsData = driver.execute_script(script)
    companyProblems = pd.DataFrame(companyProblemsData, columns = ['Id','Title','Tags','Acceptance','Difficulty','Frequency','Link'])
    companyProblems = companyProblems.set_index('Id')
    return companyProblems

In [12]:
def openAndExportAllCompanyProblems():
    startTime = time.time()
    currentTime = startTime
    optionNames = ['6_months', '1_year', '2_years', 'All_time']
    optionTotalCount = len(optionNames)
    companiesCount = len(allCompanies)
    currentTime = logTime(startTime, currentTime); print("Total Companies: %d" % companiesCount)
    for i in range(companiesCount):
        company = allCompanies.iloc[i]
        companyName = company['Name'].replace(' ','_')
        currentTime = logTime(startTime, currentTime); print("Processing %s (%d/%d) (%d%%)" % (companyName, i+1,companiesCount,((i+1)*100/companiesCount)))
        driver.get(company['Link'])
        WebDriverWait(driver, timeout=30).until(lambda x: x.find_element_by_id('react-select-2--value-item').is_displayed())
        driver.find_element_by_css_selector('input[type=checkbox]').click()
        driver.find_element_by_css_selector('.reactable-column-header th:nth-child(2)').click()
        firstOption = getFirstOption()
        currentTime = logTime(startTime, currentTime); print("Periods available: %d" % (optionTotalCount- firstOption))
        currentTime = logTime(startTime, currentTime); 
        for currentOption in range(firstOption, optionTotalCount):
            optionName = optionNames[currentOption]
            selectOption(currentOption)
            companyProblems = getCompanyProblems()
            print("%s: %d | " % (optionName, len(companyProblems)), end='')
            export(companyProblems, companyName + '-' + optionName, 'Company_Problems/Period-wise/' + optionName)
            export(companyProblems, companyName + '-' + optionName, 'Company_Problems/Name-wise/' + companyName)
        print()
    currentTime = logTime(startTime, currentTime); print("Exported Problems of %d Companies (100%%)" % companiesCount)

In [13]:
def openAndExportAllInterviews():
    url = 'https://leetcode.com/explore/interview/'
    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_class_name('explore-card').is_displayed())
    script = '''
const cards = Array.from(document.querySelectorAll('.explore-card .premium')).map(e => e.closest('.explore-card'))
const interviews = [];
for (const card of cards) {
  const interview = [];
  interview.push(card.querySelector('.title').innerText.trim());
  interview.push(card.querySelector('.chapter').innerText.trim());
  interview.push(card.querySelector('.item').innerText.trim());
  interview.push(card.querySelector('a').href);
  interviews.push(interview);
}
return interviews;
    '''
    interviewsData = driver.execute_script(script)
    interviews = pd.DataFrame(interviewsData, columns = ['Title','Chapters','Problems','Link'])
    interviews.index = interviews.index + 1
    interviews.index.name = 'Serial'
    export(interviews, 'Interviews')
    print("Exported %d Interviews (100%%)" % len(interviews))
    return interviews

In [14]:
def joinProblemDetails(problem):
    problem['Id'] = None
    cols = ['Acceptance','Difficulty','Frequency','Link']
    for col in cols:
        problem[col] = None
    try:
        problem['Id'] = allProblemsByTitle.loc[problem['Problem']]['Id']
        for col in cols:
            problem[col] = allProblems.loc[problem['Id']][col]
    except:
        pass
    return problem

In [15]:
def getInterviewProblems():
    script = '''
const tables = document.getElementsByClassName('table-base');
const problems = [];
for (let i = 1; i <= tables.length; i++) {
  const table = tables[i-1];
  const chapterTitle = table.querySelector('.chapter-title').innerText.trim();
  const items = table.querySelectorAll('*[title="Coding Question"] ~ span');
  for (let j = 1; j <= items.length; j++) {
  const item = items[j-1];
    const itemTitle = item.innerText.trim();
    problems.push([i, chapterTitle, j, itemTitle]);
  }
}
return problems
    '''
    interviewProblemsData = driver.execute_script(script)
    interviewProblems = pd.DataFrame(interviewProblemsData, columns = ['ChapterSerial','Chapter','ProblemSerial','Problem'])
    interviewProblems = interviewProblems.apply(joinProblemDetails, axis=1)
    interviewProblems = interviewProblems.set_index('ChapterSerial')
    return interviewProblems

In [16]:
def openAndExportAllInterviewProblems():
    startTime = time.time()
    currentTime = startTime
    interviewsCount = len(allInterviews)
    currentTime = logTime(startTime, currentTime); print("Total interviews: %d" % interviewsCount)
    for i in range(interviewsCount):
        interview = allInterviews.iloc[i]
        interviewTitle = interview['Title'].replace(' ','_')
        currentTime = logTime(startTime, currentTime); print("Processing %s (%d/%d) (%d%%)" % (interviewTitle, i+1,interviewsCount,((i+1)*100/interviewsCount)))
        driver.get(interview['Link'])
        WebDriverWait(driver, timeout=10).until(lambda x: str(len(x.find_elements_by_css_selector('.table-base .table-item'))) == interview['Problems'])
        interviewProblems = getInterviewProblems()
        currentTime = logTime(startTime, currentTime); print("Chapters: %s | Problems: %d" % (interview['Chapters'], len(interviewProblems)))
        export(interviewProblems, interviewTitle, 'Interview_Problems')
        currentTime = logTime(startTime, currentTime); print("Exported Problems of %d Interviews (100%%)" % interviewsCount)

In [17]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [18]:
login()

In [19]:
allProblems = openAndExportAllProblems()

Exported 1579 Problems (100%)


In [20]:
allProblemsByTitle = getProblemsByTitle()

In [21]:
allCompanies = openAndExportAllCompanies()

Exported 236 Companies (100%)


In [22]:
openAndExportAllCompanyProblems()

Duration:00:00:01.00, Total:00:06:14.67] Periods available: 2
[Duration:00:00:00.00, Total:00:06:14.67] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:14.81] Processing GE_Digital (138/236) (58%)
[Duration:00:00:02.00, Total:00:06:17.02] Periods available: 2
[Duration:00:00:00.00, Total:00:06:17.02] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:17.13] Processing infosys (139/236) (58%)
[Duration:00:00:01.00, Total:00:06:19.08] Periods available: 1
[Duration:00:00:00.00, Total:00:06:19.08] All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:19.14] Processing Jane_Street (140/236) (59%)
[Duration:00:00:03.00, Total:00:06:22.23] Periods available: 2
[Duration:00:00:00.00, Total:00:06:22.23] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:22.36] Processing Machine_Zone (141/236) (59%)
[Duration:00:00:01.00, Total:00:06:24.31] Periods available: 1
[Duration:00:00:00.00, Total:00:06:24.31] All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:24.37] 

In [23]:
allInterviews = openAndExportAllInterviews()

Exported 10 Interviews (100%)


In [24]:
openAndExportAllInterviewProblems()

[Duration:00:00:00.00, Total:00:00:00.00] Total interviews: 10
[Duration:00:00:00.00, Total:00:00:00.00] Processing Google_Interview (1/10) (10%)
[Duration:00:00:03.00, Total:00:00:03.89] Chapters: 9 | Problems: 81
[Duration:00:00:00.00, Total:00:00:03.89] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:03.89] Processing Facebook (2/10) (20%)
[Duration:00:00:03.00, Total:00:00:07.58] Chapters: 9 | Problems: 79
[Duration:00:00:00.00, Total:00:00:07.58] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:07.58] Processing Microsoft (3/10) (30%)
[Duration:00:00:03.00, Total:00:00:10.62] Chapters: 8 | Problems: 58
[Duration:00:00:00.00, Total:00:00:10.62] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:10.62] Processing Amazon (4/10) (40%)
[Duration:00:00:03.00, Total:00:00:13.72] Chapters: 8 | Problems: 68
[Duration:00:00:00.00, Total:00:00:13.72] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00

In [25]:
driver.quit()